In [ ]:
import cv2
import numpy as np
import supervision as sv
from ultralytics import YOLOv10
from geopy.distance import geodesic
import random

# Constants
IMAGE_WIDTH = 4000
IMAGE_HEIGHT = 3000
FOV_HORIZONTAL = 69.4
FOV_VERTICAL = FOV_HORIZONTAL * (IMAGE_HEIGHT / IMAGE_WIDTH)  # Assuming square pixels

# Camera position and orientation from metadata
CAMERA_GPS = (29.9525417,-85.4287138888889)  # Latitude, Longitude
ABSOLUTE_ALTITUDE = 48.01  # meters
RELATIVE_ALTITUDE = 61.10  # meters
GIMBAL_YAW_DEGREE = +31.00  # degrees
GIMBAL_PITCH_DEGREE = -89.90  # degrees (camera facing down)

# Convert degrees to radians for calculations
yaw_rad = np.radians(GIMBAL_YAW_DEGREE)
pitch_rad = np.radians(GIMBAL_PITCH_DEGREE)

# Load YOLO model
model = YOLOv10("../yolos/yolov10/yolov10s_bs_16_extended/detect/train2/weights/best.pt")

# Load the dataset
dataset = sv.DetectionDataset.from_yolo(
    images_directory_path="../data/images/test",
    annotations_directory_path="../data/labels/test",
    data_yaml_path="../yolos/dataset.yaml"
)

# Annotators
bounding_box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

# Select a random image from the dataset
#random_image = random.choice(list(dataset.images.keys()))
#random_image = dataset.images[random_image]

print(list(dataset.images.keys()))

random_image = dataset.images["../data/images/test/10808.jpg"]


# Calculate the scaling factor
#scaling_factor = min(1024 / random_image.shape[1], 1024 / random_image.shape[0])

# Resize the image while maintaining aspect ratio
#new_size = (int(random_image.shape[1] * scaling_factor), int(random_image.shape[0] * scaling_factor))
#random_image = cv2.resize(random_image, new_size, interpolation=cv2.INTER_AREA)

# Get predictions
results = model(source=random_image, conf=0.25)[0]
detections = sv.Detections.from_ultralytics(results)

# Display detections
print(detections)

# Annotate image
annotated_image = bounding_box_annotator.annotate(
    scene=random_image, detections=detections)
annotated_image = label_annotator.annotate(
    scene=annotated_image, detections=detections)

# Plot the annotated image
sv.plot_image(annotated_image)

# Function to calculate ground distance from pixel distance
def calculate_ground_distance(altitude, fov_deg, image_dimension, pixel_distance):
    # Convert FOV to radians
    fov_rad = np.radians(fov_deg)
    # Calculate ground distance
    ground_distance = (2 * altitude * np.tan(fov_rad / 2)) * (pixel_distance / image_dimension)
    return ground_distance

# Function to get GPS coordinates from offsets
def get_gps_coordinates(lat, lon, north_offset, east_offset):
    # Calculate new latitude and longitude
    new_location = geodesic(meters=north_offset).destination((lat, lon), 0)
    new_location = geodesic(meters=east_offset).destination(new_location, 90)
    return new_location.latitude, new_location.longitude

building_locations = list()
# Process each detection
for i, box in enumerate(detections.xyxy):
    # Extract bounding box coordinates
    x_min, y_min, x_max, y_max = box
    
    # Calculate the center of the bounding box
    x_center = (x_min + x_max) / 2
    y_center = (y_min + y_max) / 2

    # Calculate distances
    pixel_distance_x = x_center - IMAGE_WIDTH / 2
    pixel_distance_y = IMAGE_HEIGHT / 2 - y_center

    # Convert pixel distances to real world distances
    ground_distance_x = calculate_ground_distance(RELATIVE_ALTITUDE, FOV_HORIZONTAL, IMAGE_WIDTH, pixel_distance_x)
    ground_distance_y = calculate_ground_distance(RELATIVE_ALTITUDE, FOV_VERTICAL, IMAGE_HEIGHT, pixel_distance_y)

    # Adjust for camera orientation (yaw)
    east_offset = ground_distance_x * np.cos(yaw_rad) - ground_distance_y * np.sin(yaw_rad)
    north_offset = ground_distance_x * np.sin(yaw_rad) + ground_distance_y * np.cos(yaw_rad)

    # Get building GPS coordinates
    building_lat, building_lon = get_gps_coordinates(CAMERA_GPS[0], CAMERA_GPS[1], north_offset, east_offset)
    building_locations.append((building_lat,building_lon))
    # Print building location
    print(f"Building {i+1}: Latitude: {building_lat}, Longitude: {building_lon}  // {building_lat}, {building_lon}")

building_locations

In [ ]:
import folium

# Create a Folium map centered at the camera's GPS position with satellite tiles
map_center = CAMERA_GPS
m = folium.Map(
    location=map_center,
    zoom_start=18,  # Increase zoom level for closer view
    tiles='Esri.WorldImagery'  # Use Esri's World Imagery for satellite view
)

# Iterate over each building's GPS coordinates and add a marker
for i, (building_lat, building_lon) in enumerate(building_locations):  # building_locations contains tuples of (latitude, longitude)
    # Determine the building's status based on class_id
    building_status = 'Damaged' if detections.class_id[i] == 1 else 'Undamaged'
    
    # Create a marker for each building
    folium.Marker(
        location=(building_lat, building_lon),
        popup=f'Building {i+1}: {building_status}',
        icon=folium.Icon(color='red' if building_status == 'Damaged' else 'green', icon='home')
    ).add_to(m)

# Display the map (Jupyter Notebook) or save to an HTML file
m.save('detected_buildings_satellite_map.html')
m

todos: 
* automate digging params from image metadata
* do it for all test images
* build and interface that you can upload a image or images and extract georeferences 
